# Group project - Summarise Month Group Data

Requirements:

imputed_dates.csv

all_metadata.csv (from Case Study 3)

In [17]:
import numpy as np
from vincenty import vincenty
import pandas as pd
from scipy import spatial
from scipy.spatial import KDTree
import math

In [27]:
crashdata = pd.read_csv('imputed_dates.csv', low_memory=False)

In [28]:
# create imputed date field
crashdata['imputed_date'] = crashdata.apply(lambda x: x['ForwardImputedDate'] if x['DateImputionAgreement'] == True else None, axis = 1)

# Select relevant columns
crashdata = crashdata[['Crash_Ref_Number','Crash_Year', 'Crash_Month', 'Crash_Day_Of_Week','Crash_Hour', 'imputed_date', 'DateImputionAgreement', 'month_group' , 'Crash_Nature', 'Crash_Type', 'Crash_Longitude_GDA94', 'Crash_Latitude_GDA94', 'Crash_Street', 'Crash_Street_Intersecting', 'Loc_Suburb', 'Loc_Local_Government_Area', 'Loc_Post_Code', 'Loc_Police_Division', 'Loc_Police_District', 'Loc_Police_Region', 'Loc_Queensland_Transport_Region', 'Crash_Roadway_Feature', 'Crash_Traffic_Control', 'Crash_Speed_Limit', 'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition', 'Crash_Lighting_Condition', 'Count_Casualty_Fatality', 'Count_Casualty_Hospitalised', 'Count_Casualty_MedicallyTreated', 'Count_Casualty_MinorInjury', 'Count_Casualty_Total', 'Count_Unit_Car', 'Count_Unit_Motorcycle_Moped', 'Count_Unit_Truck', 'Count_Unit_Bus', 'Count_Unit_Bicycle', 'Count_Unit_Pedestrian', 'Count_Unit_Other' ]]

In [30]:
print(crashdata.columns.tolist())
crashdata.head()

['Crash_Ref_Number', 'Crash_Year', 'Crash_Month', 'Crash_Day_Of_Week', 'Crash_Hour', 'imputed_date', 'DateImputionAgreement', 'month_group', 'Crash_Nature', 'Crash_Type', 'Crash_Longitude_GDA94', 'Crash_Latitude_GDA94', 'Crash_Street', 'Crash_Street_Intersecting', 'Loc_Suburb', 'Loc_Local_Government_Area', 'Loc_Post_Code', 'Loc_Police_Division', 'Loc_Police_District', 'Loc_Police_Region', 'Loc_Queensland_Transport_Region', 'Crash_Roadway_Feature', 'Crash_Traffic_Control', 'Crash_Speed_Limit', 'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition', 'Crash_Lighting_Condition', 'Count_Casualty_Fatality', 'Count_Casualty_Hospitalised', 'Count_Casualty_MedicallyTreated', 'Count_Casualty_MinorInjury', 'Count_Casualty_Total', 'Count_Unit_Car', 'Count_Unit_Motorcycle_Moped', 'Count_Unit_Truck', 'Count_Unit_Bus', 'Count_Unit_Bicycle', 'Count_Unit_Pedestrian', 'Count_Unit_Other']


,Crash_Ref_Number,Crash_Year,Crash_Month,Crash_Day_Of_Week,Crash_Hour,imputed_date,DateImputionAgreement,month_group,Crash_Nature,Crash_Type,...,Count_Casualty_MedicallyTreated,Count_Casualty_MinorInjury,Count_Casualty_Total,Count_Unit_Car,Count_Unit_Motorcycle_Moped,Count_Unit_Truck,Count_Unit_Bus,Count_Unit_Bicycle,Count_Unit_Pedestrian,Count_Unit_Other
0,1.0,2001,January,Monday,6,None,False,1,Head-on,Multi-Vehicle,...,0,0,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2.0,2001,January,Wednesday,9,None,False,1,Angle,Multi-Vehicle,...,0,0,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,2001,January,Thursday,8,None,False,1,Rear-end,Multi-Vehicle,...,0,1,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,2001,January,Sunday,8,None,False,1,Hit object,Single Vehicle,...,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,2001,January,Tuesday,16,None,False,1,Overturned,Single Vehicle,...,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# Functions to summarise crash data by the number of attributed vs non-attributed  dates

def summarise_dates_by_year(group_column, dataset):
    initial_group = dataset.groupby([group_column, 'Crash_Year', 'DateImputionAgreement'])['DateImputionAgreement'].count()
    date_groups = initial_group.unstack()
    date_groups.columns = ['no_date_allocated', 'date_allocated']
    date_groups[np.isnan(date_groups)]= 0
    date_groups.no_date_allocated = date_groups.no_date_allocated.astype(int)
    date_groups.date_allocated = date_groups.date_allocated.astype(int)
    date_groups['total_events'] = date_groups['no_date_allocated'] + date_groups['date_allocated']
    date_groups['percent_allocated'] = date_groups['date_allocated'] / date_groups['total_events']
    return date_groups

def summarise_dates_all(group_column, dataset):
    initial_group = dataset.groupby([group_column, 'DateImputionAgreement'])['DateImputionAgreement'].count()
    date_groups = initial_group.unstack()
    date_groups.columns = ['no_date_allocated', 'date_allocated']
    date_groups[np.isnan(date_groups)]= 0
    date_groups.no_date_allocated = date_groups.no_date_allocated.astype(int)
    date_groups.date_allocated = date_groups.date_allocated.astype(int)
    date_groups['total_events'] = date_groups['no_date_allocated'] + date_groups['date_allocated']
    date_groups['percent_allocated'] = date_groups['date_allocated'] / date_groups['total_events']
    return date_groups

def summarise_dates_select_year(group_column, year, dataset):
    initial_group = dataset[dataset['Crash_Year'] == year].groupby([group_column, 'DateImputionAgreement'])['DateImputionAgreement'].count()
    date_groups = initial_group.unstack()
    date_groups.columns = ['no_date_allocated', 'date_allocated']
    date_groups[np.isnan(date_groups)]= 0
    date_groups.no_date_allocated = date_groups.no_date_allocated.astype(int)
    date_groups.date_allocated = date_groups.date_allocated.astype(int)
    date_groups['total_events'] = date_groups['no_date_allocated'] + date_groups['date_allocated']
    date_groups['percent_allocated'] = date_groups['date_allocated'] / date_groups['total_events']
    return date_groups

#summarise_dates_by_year('Loc_Police_Division', crashdata)
#summarise_dates_all('Loc_Police_Division', crashdata)
#summarise_dates_select_year('Loc_Police_Division', 2010, crashdata)
#pd_date_groups_2014.to_csv('pd_date_groups_2014.csv', index=False)
#pd_date_groups_2014.sort_values(by=['percent_allocated'], ascending=False)

In [43]:
# Remove rows from regions with no matches or few accidents

cleansed_crashdata = crashdata
 
# Cleanse local police divisions
date_divisions = summarise_dates_all('Loc_Police_Division', cleansed_crashdata)
no_date_divisions = date_divisions[date_divisions.percent_allocated < .3]
no_date_divisions = no_date_divisions.index.values.tolist()
cleansed_crashdata = cleansed_crashdata[~crashdata['Loc_Police_Division'].isin(no_date_divisions)]
cleansed_crashdata
 
# Cleanse suburbs
suburb_divisions = summarise_dates_all('Loc_Suburb', cleansed_crashdata)
no_date_suburbs = suburb_divisions[suburb_divisions.percent_allocated < .3]
no_date_suburbs = no_date_suburbs.index.values.tolist()
cleansed_crashdata = cleansed_crashdata[~crashdata['Loc_Suburb'].isin(no_date_suburbs)]
rare_suburbs = suburb_divisions[suburb_divisions.total_events < 20]
rare_suburbs = rare_suburbs.index.values.tolist()
cleansed_crashdata = cleansed_crashdata[~crashdata['Loc_Suburb'].isin(rare_suburbs)]


# Cleanse suburbs with 'unknown' local police division
cleansed_crashdata[cleansed_crashdata['Loc_Police_Division'] == 'Unknown'].Loc_Suburb
cleansed_crashdata = cleansed_crashdata[cleansed_crashdata['Loc_Police_Division'] != 'Unknown']

cleansed_crashdata.to_csv("cleansed_crashdata.csv", index = False)

cleansed_crashdata




/opt/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/opt/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,Crash_Ref_Number,Crash_Year,Crash_Month,Crash_Day_Of_Week,Crash_Hour,imputed_date,DateImputionAgreement,month_group,Crash_Nature,Crash_Type,...,Count_Casualty_MedicallyTreated,Count_Casualty_MinorInjury,Count_Casualty_Total,Count_Unit_Car,Count_Unit_Motorcycle_Moped,Count_Unit_Truck,Count_Unit_Bus,Count_Unit_Bicycle,Count_Unit_Pedestrian,Count_Unit_Other
0,1.0,2001,January,Monday,6,None,False,1,Head-on,Multi-Vehicle,...,0,0,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2.0,2001,January,Wednesday,9,None,False,1,Angle,Multi-Vehicle,...,0,0,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,2001,January,Thursday,8,None,False,1,Rear-end,Multi-Vehicle,...,0,1,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,2001,January,Sunday,8,None,False,1,Hit object,Single Vehicle,...,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6.0,2001,January,Wednesday,9,None,False,1,Angle,Multi-Vehicle,...,0,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
6,7.0,2001,January,Wednesday,17,None,False,1,Hit object,Single Vehicle,...,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8.0,2001,January,Thursday,9,None,False,1,Hit pedestrian,Hit pedestrian,...,1,0,1,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8,9.0,2001,January,Sunday,15,None,False,1,Angle,Multi-Vehicle,...,0,0,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9,10.0,2001,January,Tuesday,15,None,False,1,Angle,Multi-Vehicle,...,0,0,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
10,11.0,2001,January,Wednesday,19,None,False,1,Hit object,Single Vehicle,...,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
summarise_dates_by_year('Loc_Local_Government_Area', cleansed_crashdata).unstack().percent_allocated

Crash_Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Loc_Local_Government_Area,,,,,,,,,,,,,,,,,,
Brisbane City,0.806137,0.786970,0.814552,0.845081,0.812570,0.783761,0.815848,0.792640,0.770463,0.765159,0.556410,0.549223,0.556254,0.556244,0.543014,0.550601,0.540961,0.533514
Bundaberg Region,0.917219,0.931343,0.993443,0.931953,0.992147,0.924012,0.989189,0.994366,0.950820,0.870968,0.610778,0.493902,0.735751,0.578947,0.277027,0.616352,0.372881,0.206897
Cairns Region,0.821839,0.771777,0.869281,0.879185,0.884211,0.881997,0.921626,0.865559,0.861386,0.868739,0.470383,0.545455,0.602787,0.563107,0.484099,0.417323,0.445255,0.403226
Fraser Coast Region,0.478405,0.561404,0.626741,0.774359,0.836735,0.704835,0.593350,0.712895,0.750000,0.799517,0.054054,0.244565,0.138298,0.266010,0.091398,0.205405,0.182320,0.139241
Gladstone Region,0.468085,0.509202,0.394558,0.461078,0.548387,0.820225,0.720000,0.471503,0.561290,0.620112,0.232558,0.177570,0.215686,0.246753,0.000000,0.190476,0.122807,0.297297
Gold Coast City,0.813911,0.859443,0.825712,0.826889,0.830501,0.769308,0.772875,0.856836,0.826737,0.780220,0.685693,0.530176,0.578500,0.685157,0.557153,0.633970,0.652840,0.654206
Gympie Region,0.601990,0.804878,0.658385,0.522222,0.792135,0.641618,0.806034,0.874346,0.396648,0.831325,0.204819,0.294118,0.337349,0.177083,0.292135,0.329787,0.266667,0.000000
Ipswich City,0.878547,0.921951,0.919955,0.822695,0.856517,0.890825,0.913319,0.913741,0.943580,0.899103,0.456290,0.463357,0.622685,0.655779,0.500000,0.703947,0.691837,0.568889
Livingstone Shire,0.666667,0.600000,0.555556,1.000000,0.545455,0.833333,0.769231,0.800000,0.777778,0.727273,0.285714,0.333333,0.142857,0.000000,0.000000,0.222222,0.000000,0.000000


In [42]:
crashmonthsummary = cleansed_crashdata.groupby('month_group',as_index=False).aggregate({'DateImputionAgreement': lambda x: list(x)[0], \
                                                          'Crash_Ref_Number': ['count'], \
                                                          'Crash_Year': lambda x: list(x)[0], \
                                                          'Crash_Month': lambda x: list(x)[0], \
                                                          'Loc_Suburb': lambda x: set(x), \
                                                          'Loc_Police_Division': lambda x: set(x),\
                                                          'Loc_Police_District': lambda x: set(x),\
                                                          'Loc_Police_Region': lambda x: set(x),\
                                                          'Loc_Queensland_Transport_Region': lambda x: set(x),\
                                                          
                                                          })
crashmonthsummary.columns = ['month_group', 'dates_imputed', 'total_events', 'Crash_Year', 'Crash_Month', 'Loc_Suburb', 'Loc_Police_Division', 'Loc_Police_District', 'Loc_Police_Region', 'Loc_Queensland_Transport_Region']
crashmonthsummary.to_csv('crash_month_group_summary.csv', index=False)
crashmonthsummary

,month_group,dates_imputed,total_events,Crash_Year,Crash_Month,Loc_Suburb,Loc_Police_Division,Loc_Police_District,Loc_Police_Region,Loc_Queensland_Transport_Region
0,1,False,10,2001,January,"{Runcorn, Sunnybank, Acacia Ridge, Archerfield...","{Moorooka, Acacia Ridge, Upper Mount Gravatt}",{South Brisbane},{Brisbane},{SEQ South}
1,2,True,18,2001,February,"{Rocklea, Runcorn, Sunnybank, Acacia Ridge, Co...","{Calamvale, Moorooka, Acacia Ridge, Upper Moun...",{South Brisbane},{Brisbane},{SEQ South}
2,3,True,29,2001,March,"{Runcorn, Sunnybank, Acacia Ridge, Archerfield...","{Calamvale, Moorooka, Acacia Ridge, Upper Moun...",{South Brisbane},{Brisbane},{SEQ South}
3,4,False,14,2001,April,"{Runcorn, Sunnybank, Acacia Ridge, Coopers Pla...","{Calamvale, Acacia Ridge, Upper Mount Gravatt,...",{South Brisbane},{Brisbane},{SEQ South}
4,5,False,24,2001,May,"{Runcorn, Sunnybank, Acacia Ridge, Coopers Pla...","{Acacia Ridge, Upper Mount Gravatt}",{South Brisbane},{Brisbane},{SEQ South}
5,6,True,24,2001,June,"{Runcorn, Sunnybank, Acacia Ridge, Calamvale, ...","{Calamvale, Acacia Ridge, Upper Mount Gravatt}",{South Brisbane},{Brisbane},{SEQ South}
6,7,False,14,2001,July,"{Runcorn, Sunnybank, Acacia Ridge, Coopers Pla...","{Calamvale, Moorooka, Acacia Ridge, Upper Moun...",{South Brisbane},{Brisbane},{SEQ South}
7,8,True,16,2001,August,"{Sunnybank, Acacia Ridge, Sunnybank Hills, Coo...","{Calamvale, Moorooka, Acacia Ridge, Upper Moun...",{South Brisbane},{Brisbane},{SEQ South}
8,9,False,16,2001,September,"{Rocklea, Sunnybank, Acacia Ridge, Calamvale, ...","{Moorooka, Acacia Ridge, Calamvale, Upper Moun...",{South Brisbane},{Brisbane},{SEQ South}
9,10,True,24,2001,October,"{Rocklea, Sunnybank, Acacia Ridge, Sunnybank H...","{Moorooka, Acacia Ridge, Upper Mount Gravatt}",{South Brisbane},{Brisbane},{SEQ South}


## Join up with nearest BOM station

In [44]:
# Based on this tutorial: https://www.timvink.nl/closest-coordinates/

station_locations = pd.read_csv("all_metadata.csv")
station_locations = station_locations.rename(columns={'latitude': 'bom_station_latitude',
                                 'longitude': 'bom_station_longitude',
                                 'name': 'bom_station_name',
                                 'elevation': 'bom_station_elevation',
                                 'filename' : 'bom_station_filename',
                                 'id': 'bom_station_id'})

#cleansed_crashdata_stations = cleansed_crashdata.iloc[1:10000].copy(deep = True)
cleansed_crashdata_stations = cleansed_crashdata.copy(deep = True)


def cartesian(latitude, longitude, elevation = 0):
    # Convert to radians
    latitude = latitude * (math.pi / 180)
    longitude = longitude * (math.pi / 180)

    R = 6371 # 6378137.0 + elevation  # relative to centre of the earth
    X = R * math.cos(latitude) * math.cos(longitude)
    Y = R * math.cos(latitude) * math.sin(longitude)
    Z = R * math.sin(latitude)
    return (X, Y, Z)

locations = []

for index, row in station_locations.iterrows():
    coordinates = [row['bom_station_latitude'], row['bom_station_longitude']]
    cartesian_coord = cartesian(*coordinates)
    locations.append(cartesian_coord)
    
tree = spatial.KDTree(locations)

def nearest_station(latitude, longitude):
    cartesian_coord = cartesian(latitude, longitude)
    nearest = tree.query([cartesian_coord], p = 2)
    return station_locations.iloc[nearest[1][0]]['bom_station_id']

cleansed_crashdata_stations['nearest_bom_st_id'] = cleansed_crashdata_stations.apply(lambda x: nearest_station(x['Crash_Latitude_GDA94'], x['Crash_Longitude_GDA94']), axis = 1)
cleansed_crashdata_stations = pd.merge(cleansed_crashdata_stations, station_locations, left_on = 'nearest_bom_st_id', right_on = 'bom_station_id', how = 'left')
cleansed_crashdata_stations['nearest_bom_st_distance_km'] = cleansed_crashdata_stations.apply(lambda x: vincenty((x['Crash_Latitude_GDA94'], x['Crash_Longitude_GDA94']), (x['bom_station_latitude'], x['bom_station_longitude']), miles=False), axis = 1)

#print(type(cleansed_crashdata_stations))
cleansed_crashdata_stations

,Crash_Ref_Number,Crash_Year,Crash_Month,Crash_Day_Of_Week,Crash_Hour,imputed_date,DateImputionAgreement,month_group,Crash_Nature,Crash_Type,...,Count_Unit_Pedestrian,Count_Unit_Other,nearest_bom_st_id,bom_station_filename,bom_station_latitude,bom_station_elevation,bom_station_id,bom_station_longitude,bom_station_name,nearest_bom_st_distance_km
0,1.0,2001,January,Monday,6,None,False,1,Head-on,Multi-Vehicle,...,0.0,0.0,40244.0,40244_UQ.dat,-27.5756,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,1.336785
1,2.0,2001,January,Wednesday,9,None,False,1,Angle,Multi-Vehicle,...,0.0,0.0,40244.0,40244_UQ.dat,-27.5756,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,2.344483
2,3.0,2001,January,Thursday,8,None,False,1,Rear-end,Multi-Vehicle,...,0.0,0.0,40244.0,40244_UQ.dat,-27.5756,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,2.639759
3,4.0,2001,January,Sunday,8,None,False,1,Hit object,Single Vehicle,...,0.0,0.0,40244.0,40244_UQ.dat,-27.5756,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,3.423516
4,6.0,2001,January,Wednesday,9,None,False,1,Angle,Multi-Vehicle,...,0.0,0.0,40211.0,40211_UQ.dat,-27.5717,12.0,40211.0,153.0078,ARCHERFIELD AIRPORT,0.898300
5,7.0,2001,January,Wednesday,17,None,False,1,Hit object,Single Vehicle,...,0.0,0.0,40244.0,40244_UQ.dat,-27.5756,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,1.382132
6,8.0,2001,January,Thursday,9,None,False,1,Hit pedestrian,Hit pedestrian,...,1.0,0.0,40211.0,40211_UQ.dat,-27.5717,12.0,40211.0,153.0078,ARCHERFIELD AIRPORT,1.513006
7,9.0,2001,January,Sunday,15,None,False,1,Angle,Multi-Vehicle,...,0.0,0.0,40244.0,40244_UQ.dat,-27.5756,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,3.114004
8,10.0,2001,January,Tuesday,15,None,False,1,Angle,Multi-Vehicle,...,0.0,0.0,40211.0,40211_UQ.dat,-27.5717,12.0,40211.0,153.0078,ARCHERFIELD AIRPORT,1.283760
9,11.0,2001,January,Wednesday,19,None,False,1,Hit object,Single Vehicle,...,0.0,0.0,40211.0,40211_UQ.dat,-27.5717,12.0,40211.0,153.0078,ARCHERFIELD AIRPORT,1.176132


In [46]:
cleansed_crashdata_stations.to_csv("cleansed_crashdata_stations.csv", index=False)

In [48]:
print(type(cleansed_crashdata_stations['imputed_date'][0]))

<class 'NoneType'>
